In [46]:
from collections.abc import MutableMapping
from typing import Any


class ReporterItem:
    name: str
    value: Any
    units: str
    desc: str

    def __init__(self, name: str, value_args) -> None:
        self.name = name
        if isinstance(value_args, tuple):
            # Decode extra information
            self.value, self.units, self.desc = self._decode(value_args)
        else:
            self.value, self.units, self.desc = value_args, "", ""

    def __str__(self) -> str:
        str_out = f"{self.name}: {self.value}"
        if self.units:
            str_out += f" {self.units}"     
        if self.desc:
            str_out += f", {self.desc}"
        return str_out

    def __repr__(self) -> str:
        return f"ReporterItem({self.name}, {self.value}, {self.units}, {self.desc})"

    @staticmethod
    def _decode(value_args) -> tuple[Any, str, str]:
        # Guarantee at least 3 elements
        value_args = list(value_args) + ['', '']
        value, unit, desc = value_args[:3]
        if not isinstance(unit, str):
            unit = str(unit)
        if not isinstance(unit, str):
            desc = str(desc)
        return value, unit, desc


class Reporter(MutableMapping):
    subsection: str

    def __init__(self, subsection: str = '') -> None:
        self._item_dict = dict()
        self.subsection = subsection

    # ===========================
    # =   Implement Abstracts   =
    # ===========================

    def __getitem__(self, key) -> Any:
        item: ReporterItem = self._item_dict.__getitem__(key)
        return item.value

    def __setitem__(self, key, value) -> None:
        self._item_dict.__setitem__(key, ReporterItem(key, value))

    def __delitem__(self, key) -> None:
        self._item_dict.__delitem__(key)

    def __len__(self) -> int:
        return self._item_dict.__len__()

    def __iter__(self):
        ...

    def update(*args, **kwargs) -> None:
        raise NotImplementedError

    def __str__(self) -> str:
        item_strings = "\n".join([item.__str__() for item in self._item_dict.values()])
        if self.subsection:
            return self.subsection + "\n" + item_strings
        else:
            return item_strings

    def __repr__(self) -> str:
        return self.__str__()


In [47]:
import numpy as np

r = Reporter('Section 1')
r['x'] = 12.0, 'm', 'something'
r['y'] = 12.9, 'Gm', 'something else'
r['pi'] = np.pi, '', 'pi'
r['v'] = np.asfarray([1, 2, 3, 4]), 'J', 'your mom'


r

Section 1
x: 12.0 m, something
y: 12.9 Gm, something else
pi: 3.141592653589793, pi
v: [1. 2. 3. 4.] J, your mom